In [1]:
import pandas as pd
import numpy as np
from scipy.stats import pearsonr
import csv
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel

In [5]:
train_all=pd.read_csv('../alldata/si630w22-hw3-train.csv')
data_all=pd.read_csv('../alldata/si630w22-hw3-data.csv')
dev_all=pd.read_csv('../alldata/si630w22-hw3-dev.csv')

In [32]:
# group_ids=['01','02','03','04','05']
# group_ids=['06']
group_ids=['07','08','09','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25']
# '07','08','09','10','11','12','13','14','15',

In [8]:

def process_data(group_id):
    train_df=train_all[train_all['group']!='group_'+group_id]
    train_df_agg=train_df.groupby(['id'])[['id','rating']].mean().reset_index()
    train_df_agg=train_df_agg.rename(columns={'id':'question_id'})
    train_df_agg=pd.merge(train_df_agg,data_all,on='question_id')
    dev_exclude=dev_all[dev_all['group']=='group_'+group_id]
    dev1=dev_all[dev_all['group']!='group_'+group_id]
    dev2=pd.merge(dev_exclude,dev_all,on='id').drop(columns=['annotator_id_x','rating_x','group_x'])
    dev2=dev2.rename(columns={'annotator_id_y':'annotator_id','rating_y':'rating','group_y':'group','id':'question_id'})

    dev3=dev2[dev2['group']!='group_'+group_id]
    dev2=dev2[dev2['group']=='group_'+group_id]
    annotation_list=list(set(dev3['question_id'].tolist()))
    # print(annotation_list)
    dev2=dev2[dev2['question_id'].isin(annotation_list)]
    dev1=dev1.rename(columns={'id':'question_id'})


    dev1=dev1.groupby(['question_id'])[['question_id','rating']].mean().reset_index()
    dev2=dev2.groupby(['question_id'])[['question_id','rating']].mean().reset_index()
    dev3=dev3.groupby(['question_id'])[['question_id','rating']].mean().reset_index()
    
    dev1=pd.merge(dev1,data_all,on='question_id')
    dev2=pd.merge(dev2,data_all,on='question_id')
    dev3=pd.merge(dev3,data_all,on='question_id')
    
    return train_df_agg,dev1,dev2,dev3

In [26]:
train_df_agg,dev1,dev2,dev3=process_data('06')
dev1

,question_id,rating,question_text,reply_id,reply_text,rlen
0,t3_n26t3x,5.000000,"Gamer girls, what are your experiences when ga...",gwhl8wk,Most males have found it refreshing that I’m t...,603
1,t3_n27j2t,4.400000,How can I deal with loneliness in a healthier ...,gwhrkod,Journal!! That’s the best thing ever. Not ever...,440
2,t3_n28ch5,3.166667,Offset panda: is it a scam?,gwila0j,"I went to the only website I could find, and i...",350
3,t3_n29tbt,3.250000,How would most people actually react to an apo...,gwi4duw,most would probably all hurry up and buy all t...,108
4,t3_n2awyq,4.200000,How has photography changed the world?,gwibgl5,It's added perspective by allowing people to s...,455
...,...,...,...,...,...,...
806,t3_np84aj,4.750000,What is the best way to hide a body?,h03oc2y,"Warm humid climate, warm water - wrap the body...",437
807,t3_npe9n2,5.000000,ELI5 : Why does the area around the wound feel...,h04l2ng,"Damaged / inflamed tissue releases histamines,...",342
808,t3_npex8i,4.600000,Is it wrong or/and bad to write about the same...,h04o1o3,Authors have their niche and I think readers f...,897
809,t3_npf5lq,2.833333,"Retail worker, what are your horror stories?",h04r6dx,Someone pooped in a shopping cart.,34


In [10]:
class Example:
    def __init__(self,question_id,question,answer,label=None):
        self.question_id=question_id
        self.question=question
        self.answer=answer
        self.label=label

In [11]:
def to_input(df):
    question_list=[]
    input_list=[]
    for idx,row in df.iterrows():
#         user_id=row['annotator_id']
        question_id=row['question_id']
        if 'rating' in df.columns:
#             print(row['rating'])
            if row['rating']==1 or row['rating']==2 or row['rating']==3 or row['rating']==4 or row['rating']==5:
                rating=int(row['rating'])-1
            else:
                continue
        else:
            rating=None
        question=row['question_text']
        answer=row['reply_text']
#         print(question_id,rating,question,answer)
        
        example=Example(question_id,question,answer,rating)
        input_list.append(example)
    return input_list

In [12]:
def to_sequence(input_list,tokenizer,max_length=128):
    X_list=[]
    rating_list=[]
    for example in input_list:
        x=example.question+"['[SEP]']"+example.answer
        label=example.label
        rating_list.append(label)
        X_list.append(x)
#     print("=======================")
#     print(X_list)
#     print("=======================")
    X_train=tokenizer(X_list,padding=True,truncation=True,max_length=max_length)
    X_train=dict(X_train)
    return X_train,rating_list

In [13]:
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [27]:
def to_dataset(train_df,dev_df1,dev_df2,dev_df3):
    
    tokenizer = AutoTokenizer.from_pretrained("microsoft/MiniLM-L12-H384-uncased")

    input_list=to_input(train_df)
    X_train,Y_train=to_sequence(input_list,tokenizer)
    dev1_list=to_input(dev_df1)
    X_dev1,Y_dev1=to_sequence(dev1_list,tokenizer)
    
    dev2_list=to_input(dev_df2)
    if len(dev2_list)!=0:
        X_dev2,Y_dev2=to_sequence(dev2_list,tokenizer)
    else:
        X_dev2=None
        Y_dev2=[]
        
    dev3_list=to_input(dev_df3)
    if len(dev3_list)!=0:
        X_dev3,Y_dev3=to_sequence(dev3_list,tokenizer)
    else:
        X_dev3=None
        Y_dev3=[]
#     print(X_dev3)
    
    max_length=128
    train_dataset = Dataset(X_train,Y_train)
    val_dataset1 = Dataset(X_dev1,Y_dev1)
    val_dataset2 = Dataset(X_dev2,Y_dev2)
    val_dataset3 = Dataset(X_dev3,Y_dev3)
    
    return train_dataset,[val_dataset1,val_dataset2,val_dataset3],[[X_dev1,Y_dev1],[X_dev2,Y_dev2],[X_dev3,Y_dev3]]

In [18]:
from transformers import TrainingArguments, Trainer

device=torch.device("cuda" if torch.cuda.is_available else "cpu")

class Regression(nn.Module):
    def __init__(self):
        super(Regression, self).__init__()
        self.miniLM=AutoModel.from_pretrained("microsoft/MiniLM-L12-H384-uncased")
        self.regressor=nn.Linear(384,1)
    def forward(self,input_ids,attention_mask,token_type_ids,labels=None):
        input_ids=input_ids.to(device)
        attention_mask=attention_mask.to(device)
        token_type_ids=token_type_ids.to(device)
        output=self.miniLM(input_ids,attention_mask,token_type_ids)
#         print("output",output)
#         print("output type:",type(output))
#         print("output[1]:",output[1])
#         print("output[1].shape:",output[1].shape)
        pred=self.regressor(output[1])
#         print("pred:",pred)
        return pred

class RevisedTrainer(Trainer):
    def compute_loss(self,model,inputs,return_outputs=False):
        label=torch.FloatTensor(list(inputs["labels"])).to(device)
#         print(ratings)
        outputs=model(**inputs)
        loss_func=nn.MSELoss()
        loss=loss_func(outputs.squeeze(1).float(),label)
        return (loss,outputs) if return_outputs else loss


def compute_metrics(p):
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)

    return {"accuracy": accuracy}


In [19]:


def calculate_pearson(input_list,label_list,max_length,trainer):
#     print(input_list['input_ids'])
    data_unlabeled = Dataset(input_list,label_list)
    pred_list=[]
    dataloader=torch.utils.data.DataLoader(data_unlabeled)
    for step,data in enumerate(dataloader):
        output=model1(data['input_ids'],data['token_type_ids'],data['attention_mask'])
        pred_list.append(output.tolist()[0][0])
        
#     pred=trainer.predict(data_unlabeled)

#     print(pred_list)
    print(len(pred_list))
    print(len(label_list))
    
    pr,_=pearsonr(pred_list,label_list)
#     print(pr)
    return pr

In [33]:
all_pr=[]
max_length=128

for group in group_ids:
    train_df,dev_df1,dev_df2,dev_df3=process_data(group)
#     print(train_df.head())
#     break
    train_dataset,eval_dataset_list,all_eval_list=to_dataset(train_df,dev_df1,dev_df2,dev_df3)
    
#     print(all_eval_list[0]==all_eval_list[1])
    
    print("=================group:",group,"===================")
    pr_list=[]
    for i in range(3):
        print("============",i,"=============")
        eval_dataset=eval_dataset_list[i]
        [eval_list,label_list]=all_eval_list[i]

        
        args = TrainingArguments(output_dir="ouput_part3",
        evaluation_strategy="epoch",
                            per_device_train_batch_size=8,
                            per_device_eval_batch_size=8,
                            num_train_epochs=5)

        model1=Regression()
        trainer1 = RevisedTrainer(
            model=model1.to(device),
            args=args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset

        )
        
        # Train pre-trained model
        trainer1.train()
        pr=calculate_pearson(eval_list,label_list,max_length,trainer1)
        print("pearson score:",pr)
        pr_list.append(pr)
    all_pr.append(pr_list)
    
    if group=='05' or group=='10' or group=='15' or group=='25':
        df=pd.DataFrame(all_pr,columns=['A','B','C'])
        filename='group'+group+"checkpoint.csv"
        df.to_csv(filename)
    

loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /home/zihuiliu/.cache/huggingface/transformers/ceb753d3f27a8c0d09184f35884666cda91b8ae610cd2a54d89793ac7663f1f9.13815020fd994b27db9974c0ce0ec4c47dfac6c8f11bf1a35a0a06d5b165665a
Model config BertConfig {
  "_name_or_path": "microsoft/MiniLM-L12-H384-uncased",
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 384,
  "initializer_range": 0.02,
  "intermediate_size": 1536,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.17.0",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/vocab.txt from cache at

=================group: 07 ===================
============ 0 =============


loading weights file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/pytorch_model.bin from cache at /home/zihuiliu/.cache/huggingface/transformers/b774244369e464de2c660477b70bae7c3223fa7250aa1c8fc0b0f037ed58418a.087808d17814e241e9352c5ce0fea1a7d05e5b0f020d44b42b5f05922e96c923
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
/home/zihuiliu/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1152

Epoch,Training Loss,Validation Loss
1,No log,0.701066
2,No log,0.573868
3,No log,0.572334
4,0.758000,0.635622
5,0.758000,0.573225


***** Running Evaluation *****
  Num examples = 241
  Batch size = 8
***** Running Evaluation *****
  Num examples = 241
  Batch size = 8
***** Running Evaluation *****
  Num examples = 241
  Batch size = 8
Saving model checkpoint to ouput_part3/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 241
  Batch size = 8
***** Running Evaluation *****
  Num examples = 241
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/config.json from cache at /home/zihuiliu/.cache/hugg

...

52
52
pearson score: 0.8228835444856302
============ 2 =============


loading weights file https://huggingface.co/microsoft/MiniLM-L12-H384-uncased/resolve/main/pytorch_model.bin from cache at /home/zihuiliu/.cache/huggingface/transformers/b774244369e464de2c660477b70bae7c3223fa7250aa1c8fc0b0f037ed58418a.087808d17814e241e9352c5ce0fea1a7d05e5b0f020d44b42b5f05922e96c923
All model checkpoint weights were used when initializing BertModel.

All the weights of BertModel were initialized from the model checkpoint at microsoft/MiniLM-L12-H384-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use BertModel for predictions without further training.
/home/zihuiliu/.local/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 1135

Epoch,Training Loss,Validation Loss
1,No log,1.382736
2,No log,0.998098
3,No log,0.938436
4,0.724900,0.840941
5,0.724900,0.854156


***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
Saving model checkpoint to ouput_part3/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8
***** Running Evaluation *****
  Num examples = 37
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




37
37
pearson score: 0.5191157849747964


In [ ]:
len(dic['input_ids'])

In [63]:
len(dic['token_type_ids'])

128

In [64]:
len(dic['attention_mask'])

128